In [1]:
# 필요한 라이브러리 로드
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
# 원본 데이터
raw_data = pd.read_csv("uci-secom.csv")

In [3]:
# 데이터 크기 확인
data_count = raw_data.shape[0]

# 시계열 특성 데이터라고 가정하지 않으므로 Time 속성 제거
shrinked_data = raw_data.drop(["Time"], axis=1)

# 전체의 50% 이상이 null 값을 가진 특성 제거
shrinked_data = shrinked_data.loc[:, (raw_data.isna().sum(axis=0) <= data_count * 0.5)]

# 특성의 모든 값이 상수를 가진 특성을 제거
shrinked_data = shrinked_data.loc[:, (raw_data.var(axis=0) != 0)]

# 종속 변수와 독립 변수를 분리
cause_matrix = shrinked_data.drop(["Pass/Fail"], axis=1)

# 독립 변수끼리의 상관 관계가 높은 항목을 제거
corr_matrix = cause_matrix.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

shrinked_data.drop(to_drop, axis=1, inplace=True)

shrinked_data.shape

(1567, 273)

In [5]:
# K개의 이웃을 통해 결측치를 예측
imputer = KNNImputer(n_neighbors=5)
imputed_data = pd.DataFrame(imputer.fit_transform(shrinked_data),columns = shrinked_data.columns)

In [6]:
imputed_data.isna().any()

0            False
1            False
2            False
3            False
4            False
             ...  
583          False
586          False
587          False
589          False
Pass/Fail    False
Length: 273, dtype: bool